In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
from konlpy.tag import Okt
from collections import defaultdict

In [ ]:
def make_df(df_, name):
    df = df_
    df.columns = df.iloc[5]
    df = df.iloc[6:].reset_index(drop=True)

    # 컬럼 이름 제거
    df.columns.name = None

    # 컬럼 재설정
    df = df[['발명의명칭', 'IPC분류', '출원일자', '출원인', '법적상태', '요약']]

    # IPC코드 추출
    df['IPC분류'] = df['IPC분류'].str[:4]

    # 데이터 유형 변경
    df['IPC분류'] = df['IPC분류'].astype('category')
    df['출원일자'] = pd.to_datetime(df['출원일자'], format='%Y.%m.%d')

    # 출원인 컬럼 통합
    df['출원인'] = name

    # IPC코드-기술분류 연계표 불러오기
    df_i = pd.read_excel('data/IPC코드-기술분류 연계표.xlsx')

    # IPC분류를 기준으로 기술분류 컬럼 병합
    df = pd.merge(df, df_i[['IPC분류', '기술분류']], on='IPC분류', how='left')

    # 결측치는 '기타'로 채우기
    df['기술분류'] = df['기술분류'].fillna('기타')

    # 데이터 유형 변경
    df['기술분류'] = df['기술분류'].astype('category')
    df['IPC분류'] = df['IPC분류'].astype('category')

    return df

In [ ]:
df = pd.read_csv("data/LIG넥스원.csv")
df['기술분류'].value_counts()

기술분류
항공/우주 시스템         533
정보통신 모듈 및 부품      177
기타                158
치료기기및 진단기기        119
소프트웨어             113
이동통신              112
RFID/USN           87
에너지효율 향상           81
조선/해양시스템           78
신재생에너지             71
자동차/철도차량           57
광전자 소재             48
디지털방송              46
로봇/자동화기계           45
광응용기기              42
전지                 32
디지털콘텐츠             25
중전기기               25
반도체소자및시스템          21
위성-전파              19
소성/가공/분말           19
디자인                15
산업/일반기계            13
영상/음향 기기            8
계측기기                7
스마트 그리드             6
요소부품                5
연구개발/엔지니어링 서비스      5
정밀생산기계              4
가정용기기및전자응용기기        4
ITS/텔레매틱스           4
자원                  4
고분자재료               3
화력발전                3
섬유제품                2
금속재료                2
전기전자부품              2
산업바이오               1
정밀화학                1
섬유재료                1
디스플레이               1
에너지/환경기계 시스템        1
Name: count, dtype: int64

In [ ]:
df.head()

,발명의명칭,IPC분류,출원일자,출원인,법적상태,요약,기술분류
0,위상배열 송수신 시스템의 위상 보정 제어 방법 및 그를 위한 장치,G01S,2021-12-30,LIG넥스원,공개,위상배열 송수신 시스템의 위상 보정 제어 방법 및 그를 위한 장치를 개시한다. 본 ...,항공/우주 시스템
1,능동 위상 배열 안테나의 수신 패턴 분석 방법,H04B,2021-08-31,LIG넥스원,등록,본 발명은 능동 위상 배열 안테나의 수신 패턴 측정시에 능동 위상 배열 안테나에서 ...,정보통신 모듈 및 부품
2,"유도 무기, 표적 추적 시스템 및 표적 추적 방법",G01S,2021-06-29,LIG넥스원,등록,표적 추적 시스템 및 표적 추적 방법이 개시될 수 있다. 일 실시예에 따른 표적 추...,항공/우주 시스템
3,"소형 무인기용 레이더 센서 탑재체, 원격 처리부, 탐지 시스템 및 이를 이용한 대상...",G01S,2020-05-08,LIG넥스원,등록,"소형 무인기용 레이더 센서 탑재체, 원격 처리부, 탐지 시스템 및 이를 이용한 대상...",항공/우주 시스템
4,열가소성 추진제 및 이를 제조하는 방법,C06B,2023-07-18,LIG넥스원,공개,열가소성 추진제 및 이를 제조하는 방법이 개시될 수 있다. 본 발명의 일 실시예에 ...,로봇/자동화기계
